# 🚦 SRIG — Civic Score Predictor
### Smart Road India Grid | Model 3 — Real-time Prediction
**Yash Jani | Red and White Institute, Surat**

---

Is notebook mein tum **kisi bhi driver ka data daaloge** aur model turant batayega:
- 🏅 Risk Level (PLATINUM / GOLD / SILVER / BRONZE / HIGH_RISK)
- 📊 Confidence Score
- 📈 Har class ki probability

## 📦 Step 1 — Libraries Import Karo

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

print('✅ Libraries import ho gayi!')

## 🏗️ Step 2 — Model Architecture Define Karo
(Same as training — zaroori hai load karne ke liye)

In [ ]:
class TabTransformer(nn.Module):
    def __init__(self, input_dim, num_classes, d_model=64, nhead=4, num_layers=3, dropout=0.1):
        super(TabTransformer, self).__init__()
        self.input_projection = nn.Linear(input_dim, d_model)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=nhead,
            dim_feedforward=128, dropout=dropout, batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.classifier = nn.Sequential(
            nn.LayerNorm(d_model), nn.Linear(d_model, 64),
            nn.ReLU(), nn.Dropout(dropout), nn.Linear(64, num_classes)
        )

    def forward(self, x):
        x = self.input_projection(x)
        x = x.unsqueeze(1)
        x = self.transformer(x)
        x = x.squeeze(1)
        return self.classifier(x)

print('✅ Model architecture ready!')

## 💾 Step 3 — Trained Model Load Karo

In [ ]:
checkpoint = torch.load('models/civic_score_model.pth', map_location='cpu')

le           = checkpoint['label_encoder']
scaler       = checkpoint['scaler']
classes      = checkpoint['classes']
FEATURE_COLS = checkpoint['feature_cols']

model = TabTransformer(input_dim=10, num_classes=len(classes))
model.load_state_dict(checkpoint['model_state'])
model.eval()

print(f'✅ Model load ho gaya!')
print(f'   Classes : {classes}')

## 🔮 Step 4 — Prediction Function Banao

In [ ]:
# Color mapping for each risk level
COLORS = {
    'PLATINUM': '#4FC3F7',
    'GOLD':     '#FFD700',
    'SILVER':   '#C0C0C0',
    'BRONZE':   '#CD7F32',
    'HIGH_RISK':'#FF4444'
}

EMOJIS = {
    'PLATINUM': '💎',
    'GOLD':     '🥇',
    'SILVER':   '🥈',
    'BRONZE':   '🥉',
    'HIGH_RISK':'⚠️'
}

def predict_and_show(driver_data: dict, driver_name: str = 'Driver'):
    # Features extract karo
    features = np.array([[driver_data[col] for col in FEATURE_COLS]])
    features_scaled = scaler.transform(features)
    tensor = torch.FloatTensor(features_scaled)

    # Prediction
    with torch.no_grad():
        output = model(tensor)
        probs  = torch.softmax(output, dim=1).squeeze().numpy()
        predicted_idx   = probs.argmax()
        predicted_class = classes[predicted_idx]
        confidence      = probs[predicted_idx] * 100

    # ── Terminal Output ──
    print('\n' + '='*50)
    print(f'  🚦 SRIG CIVIC SCORE — {driver_name.upper()}')
    print('='*50)
    print(f'  {EMOJIS[predicted_class]} Risk Level  : {predicted_class}')
    print(f'  📊 Confidence  : {confidence:.1f}%')
    print('\n  All Probabilities:')
    for cls, prob in zip(classes, probs):
        bar = '█' * int(prob * 25)
        print(f'  {cls:<12}: {bar:<25} {prob*100:.1f}%')
    print('='*50)

    # ── Visual Chart ──
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    fig.suptitle(f'SRIG Civic Score — {driver_name}', fontsize=14, fontweight='bold')

    # Chart 1: Probability Bar Chart
    bar_colors = [COLORS.get(c, '#888888') for c in classes]
    bars = axes[0].bar(classes, probs * 100, color=bar_colors, edgecolor='white', linewidth=1.5)
    axes[0].set_title('Risk Level Probabilities', fontsize=12)
    axes[0].set_ylabel('Probability (%)')
    axes[0].set_ylim(0, 110)
    axes[0].tick_params(axis='x', rotation=15)
    for bar, prob in zip(bars, probs):
        axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                    f'{prob*100:.1f}%', ha='center', fontsize=10, fontweight='bold')
    axes[0].grid(axis='y', alpha=0.3)

    # Chart 2: Driver Feature Profile
    feature_labels = [
        'Helmet\nCompliance', 'Signal\nViolations', 'Speed\nViolations',
        'School Zone\nViolations', 'Drunk\nDriving', 'Accident\nHistory',
        'Years\nDriving', 'Training\nCompleted', 'Complaints\nFiled', 'City\nTier'
    ]
    values = [driver_data[col] for col in FEATURE_COLS]
    bar_c  = [COLORS[predicted_class]] * len(values)
    axes[1].barh(feature_labels, values, color=bar_c, edgecolor='white')
    axes[1].set_title('Driver Feature Profile', fontsize=12)
    axes[1].set_xlabel('Value')
    axes[1].grid(axis='x', alpha=0.3)
    for i, v in enumerate(values):
        axes[1].text(v + 0.1, i, str(v), va='center', fontsize=9)

    plt.tight_layout()
    plt.savefig(f'models/prediction_{driver_name.replace(" ","_")}.png', dpi=150, bbox_inches='tight')
    plt.show()

    return predicted_class, confidence

print('✅ Prediction function ready!')

## 🧪 Step 5 — Test Cases Chalao

### Test 1 — Achha Driver (Expected: PLATINUM/GOLD)

In [ ]:
predict_and_show({
    'helmet_compliance_days': 28,
    'signal_violations':      1,
    'speed_violations':       0,
    'school_zone_violations': 0,
    'drunk_driving_cases':    0,
    'accident_history':       0,
    'years_driving':          15,
    'training_completed':     1,
    'complaints_filed':       0,
    'city_tier':              1
}, driver_name='Ramesh - Achha Driver')

### Test 2 — Average Driver (Expected: SILVER/BRONZE)

In [ ]:
predict_and_show({
    'helmet_compliance_days': 15,
    'signal_violations':      5,
    'speed_violations':       3,
    'school_zone_violations': 1,
    'drunk_driving_cases':    0,
    'accident_history':       1,
    'years_driving':          5,
    'training_completed':     0,
    'complaints_filed':       2,
    'city_tier':              2
}, driver_name='Suresh - Average Driver')

### Test 3 — Khatarnak Driver (Expected: HIGH_RISK)

In [ ]:
predict_and_show({
    'helmet_compliance_days': 3,
    'signal_violations':      15,
    'speed_violations':       12,
    'school_zone_violations': 8,
    'drunk_driving_cases':    2,
    'accident_history':       4,
    'years_driving':          2,
    'training_completed':     0,
    'complaints_filed':       8,
    'city_tier':              3
}, driver_name='Mukesh - High Risk Driver')

## ✏️ Step 6 — Apna Khud Ka Driver Test Karo!

Niche values change karo aur apna driver test karo 👇

In [ ]:
# ✏️ YAHAN VALUES CHANGE KARO
my_driver = {
    'helmet_compliance_days': 20,   # 0-30 (kitne din helmet pehna)
    'signal_violations':      3,    # 0-20 (signal kitni baar toda)
    'speed_violations':       2,    # 0-15 (speed limit kitni baar toda)
    'school_zone_violations': 0,    # 0-10 (school zone mein violation)
    'drunk_driving_cases':    0,    # 0-3  (drunk driving cases)
    'accident_history':       0,    # 0-5  (accidents)
    'years_driving':          8,    # 1-40 (kitne saal se drive kar raha hai)
    'training_completed':     1,    # 0=No, 1=Yes
    'complaints_filed':       1,    # 0-10 (complaints)
    'city_tier':              1     # 1=Metro, 2=Tier2, 3=Rural
}

predict_and_show(my_driver, driver_name='Mera Driver')